In [ ]:
# %% [markdown]
# # Análisis 1.2 - Condiciones de Frontera y Estabilidad Numérica

# %%
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Configuración
plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 6)})

# %%
def cargar_datos(archivo):
    """Carga datos de simulación FDTD"""
    with open(archivo, 'r') as f:
        content = f.read()
    
    sections = content.split('---')
    E_data = []
    for line in sections[0].strip().split('\n'):
        if line.strip() and not line.startswith('Campo'):
            E_data.append([float(x) for x in line.split()])
    
    H_data = []
    for line in sections[1].strip().split('\n'):
        if line.strip() and not line.startswith('Campo'):
            H_data.append([float(x) for x in line.split()])
    
    E = np.array(E_data)
    H = np.array(H_data)
    z = np.arange(E.shape[1])
    t = np.arange(E.shape[0])
    
    return E, H, z, t

# %% [markdown]
# ## 1.2(a) Comparación de condiciones de frontera

# %%
# Cargar datos
E_sin, H_sin, z, t = cargar_datos('resultados_sin_fronteras.txt')
E_dir, H_dir, _, _ = cargar_datos('resultados_dirichlet.txt')

# %%
# Figura 1: Comparación antes de llegar a las paredes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Tiempo antes de llegar a paredes (mitad del tiempo)
t_medio = len(t) // 2

ax1.plot(z, E_sin[t_medio, :], 'b-', label='Sin fronteras', linewidth=2)
ax1.plot(z, E_dir[t_medio, :], 'r--', label='Dirichlet (E=0)', linewidth=2)
ax1.set_xlabel('Posición z')
ax1.set_ylabel('Campo Eléctrico E')
ax1.set_title('ANTES de llegar a paredes (t = {})'.format(t_medio))
ax1.legend()
ax1.grid(True)

# Tiempo después de llegar a paredes
t_final = len(t) - 1

ax2.plot(z, E_sin[t_final, :], 'b-', label='Sin fronteras', linewidth=2)
ax2.plot(z, E_dir[t_final, :], 'r--', label='Dirichlet (E=0)', linewidth=2)
ax2.set_xlabel('Posición z')
ax2.set_ylabel('Campo Eléctrico E')
ax2.set_title('DESPUÉS de llegar a paredes (t = {})'.format(t_final))
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig('comparacion_fronteras.png', dpi=150, bbox_inches='tight')
plt.show()

# %%
# Análisis de las condicones de frontera---------------------------------------------------
print("ANÁLISIS 1.2(a): Condiciones de Frontera")
print("=" * 50)

# Energía antes de paredes
energia_sin_antes = np.sum(E_sin[t_medio, :]**2 + H_sin[t_medio, :]**2)
energia_dir_antes = np.sum(E_dir[t_medio, :]**2 + H_dir[t_medio, :]**2)

# Energía después de paredes
energia_sin_despues = np.sum(E_sin[t_final, :]**2 + H_sin[t_final, :]**2)
energia_dir_despues = np.sum(E_dir[t_final, :]**2 + H_dir[t_final, :]**2)

print(f"Energía sin fronteras - antes: {energia_sin_antes:.3f}, después: {energia_sin_despues:.3f}")
print(f"Energía Dirichlet - antes: {energia_dir_antes:.3f}, después: {energia_dir_despues:.3f}")
print(f"Conservación energía sin fronteras: {(energia_sin_despues/energia_sin_antes*100):.1f}%")
print(f"Conservación energía Dirichlet: {(energia_dir_despues/energia_dir_antes*100):.1f}%")

# %% [markdown]
# ## 1.2(b) Estabilidad numérica y condición de Courant

# %%
# Cargar datos con diferentes beta
beta_values = [0.2, 0.5, 0.8, 1.0, 1.2]
datos_beta = {}

for beta in beta_values:
    try:
        E, H, z, t = cargar_datos(f'resultados_beta_{beta}.txt')
        datos_beta[beta] = (E, H)
        print(f"Beta = {beta}: cargado correctamente")
    except:
        print(f"Beta = {beta}: archivo no encontrado")

# %%
# Figura 2: Análisis de estabilidad
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Evolución de energía
for beta, (E, H) in datos_beta.items():
    energia = np.sum(E**2 + H**2, axis=1)
    energia_norm = energia / energia[0]  # Normalizada
    ax1.plot(t, energia_norm, label=f'β = {beta}', linewidth=2)

ax1.set_xlabel('Tiempo')
ax1.set_ylabel('Energía Normalizada')
ax1.set_title('Conservación de Energía - Diferentes β')
ax1.legend()
ax1.grid(True)

# Máxima amplitud
for beta, (E, H) in datos_beta.items():
    max_amplitude = np.max(np.abs(E), axis=1)
    ax2.plot(t, max_amplitude, label=f'β = {beta}', linewidth=2)

ax2.set_xlabel('Tiempo')
ax2.set_ylabel('Máxima amplitud de |E|')
ax2.set_title('Estabilidad Numérica - Diferentes β')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig('analisis_estabilidad.png', dpi=150, bbox_inches='tight')
plt.show()

# %%
# Análisis cuantitativo de estabilidad
print("\nANÁLISIS 1.2(b): Estabilidad Numérica")
print("=" * 50)
print("Condición de Courant: β = Δt/Δz ≤ 1")

for beta, (E, H) in datos_beta.items():
    energia_inicial = np.sum(E[0, :]**2 + H[0, :]**2)
    energia_final = np.sum(E[-1, :]**2 + H[-1, :]**2)
    ratio_energia = energia_final / energia_inicial
    
    max_amp_final = np.max(np.abs(E[-1, :]))
    
    estado = " ESTABLE" if ratio_energia <= 1.1 else "INESTABLE"
    if beta > 1.0:
        estado = "VIOLA COURANT"
    
    print(f"β = {beta}: {estado}")
    print(f"   Energía final/inicial: {ratio_energia:.4f}")
    print(f"   Máxima amplitud final: {max_amp_final:.4f}")

# %% [markdown]
# ## Animación de propagación (opcional)

# %%
def crear_animacion_simple(E, H, z, t, filename):
    """Crea animación simple de la propagación"""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    line, = ax.plot(z, E[0, :], 'b-', linewidth=2)
    ax.set_xlim(z[0], z[-1])
    ax.set_ylim(np.min(E)*1.1, np.max(E)*1.1)
    ax.set_xlabel('Posición z')
    ax.set_ylabel('Campo Eléctrico E')
    ax.grid(True)
    
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
    
    def update(frame):
        line.set_ydata(E[frame, :])
        time_text.set_text(f'Tiempo: {frame}/{len(t)-1}')
        return line, time_text
    
    anim = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)
    anim.save(filename, writer='pillow', fps=20, dpi=100)
    plt.close()
    
    return anim

# Crear animación para beta estable (0.5)
if 0.5 in datos_beta:
    E, H = datos_beta[0.5]
    crear_animacion_simple(E, H, z, t, 'propagacion_estable.gif')
    print("\nAnimación de propagación estable guardada: 'propagacion_estable.gif'")

# %%
